<a href="https://colab.research.google.com/github/profsuccodifrutta/ai_act_RAG_navigator/blob/main/ai_act_RAG2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CREATION OF ENVIORMENT

In [ ]:
# install uv
!curl -LsSf https://astral.sh/uv/install.sh | sh
import sys
import os
from google.colab import userdata
sys.path.append("/root/.cargo/bin")

# configuratrion github with secrets
GIT_USER = "profsuccodifrutta"
GIT_REPO = "ai_act_RAG_navigator"

try:
    GIT_TOKEN = userdata.get('github_token')
except Exception:
    print("ERROR")
    sys.exit(1)

# clone repo
if not os.path.exists(GIT_REPO):
    !git clone https://{GIT_TOKEN}@github.com/{GIT_USER}/{GIT_REPO}.git
else:
    print(f"folder {GIT_REPO} already exists.")

%cd {GIT_REPO}

!git remote set-url origin https://{GIT_TOKEN}@github.com/{GIT_USER}/{GIT_REPO}.git

if not os.path.exists("pyproject.toml"):
    !uv init
else:
    print("uv already initialized.")

# add libraries
!uv add langchain langchain-community langchain-google-genai faiss-cpu pymupdf spacy python-dotenv
!uv add langchain-text-splitters langchain-core

# install on colab
!uv pip install --system langchain langchain-community langchain-google-genai faiss-cpu pymupdf spacy python-dotenv
!uv pip install --system langchain-text-splitters langchain-core

print("\nEnviorment configuration completed")

In [3]:
# configuration
!git config --global user.email "matteo@drezzo.ch"
!git config --global user.name "profsuccodifrutta"
!git branch -M main

# LOADING LIBRARIES

In [ ]:
!pip install langchain langchain-community langchain-classic rank_bm25 faiss-cpu

In [4]:
from google.colab import drive
import os
import requests
import shutil
import fitz
import re
import pandas as pd
import matplotlib.pyplot as plt
import gc
import random
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
from spacy import displacy
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import plotly.graph_objects as go
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import plotly.io as pio
import plotly.express as px
import numpy as np
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score, roc_auc_score
from scipy.special import softmax
from langchain_classic.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from collections import defaultdict

# SETTING UP DRIVE

Creates a local data/ directory within the repository, and loads the official 144-page EU AI Act PDF. It also prepares a dedicated folder on Drive to store the future FAISS vector index

In [5]:
drive.mount('/content/drive')

# local env set up
data_dir = "data"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    print(f"Folder '{data_dir}' created.")

# path configuration
drive_pdf_path = "/content/drive/MyDrive/compliance_and_regulation_rag/regulation_pdfs/ai_act_2024.pdf"

local_pdf_path = os.path.join(data_dir, "eu_ai_act.pdf") # local destination

# Copy file from Drive to Local Data folder
if os.path.exists(drive_pdf_path):
    shutil.copy(drive_pdf_path, local_pdf_path)
    print(f"File copied from Drive to: {local_pdf_path} ({os.path.getsize(local_pdf_path)} bytes)")
else:
    print(f"Error: File NOT found on Drive at {drive_pdf_path}")

# persistence folder for FAISS index
drive_persistence_path = "/content/drive/MyDrive/compliance_and_regulation_rag/ai_act_rag_index"
if not os.path.exists(drive_persistence_path):
    os.makedirs(drive_persistence_path)
    print("Folder for index persistence ready on Drive.")

Mounted at /content/drive
File copied from Drive to: data/eu_ai_act.pdf (2583319 bytes)


# PRE-PROCESSING and EDA

The parse_ai_act_robust() function acts as a highly specialized document parser designed to extract the EU AI Act into clean, semantically isolated legal units. First, it reads the PDF page by page, stripping away formatting noise like document links, page numbers, and line-break hyphens. It employs a multiline regex strategy to detect and truncate footnote blocks at the bottom of the pages; this prevents those footnotes from being falsely extracted as standalone Recitals or corrupting the flow of the main text. After sanitizing the pages, it groups them into "RECITAL", "ARTICLE", and "ANNEX" buckets based on their structural locations in the document. Finally, it stitches the text back together and uses strict, newline-aware regular expressions to split the document only at true section headers, entirely ignoring inline cross-references (like "in accordance with Article 99"). This guarantees that every row in the resulting dataframe represents a single, complete, and unbroken legal concept perfect for a RAG system.

In [ ]:
def parse_ai_act_robust(pdf_path):
    doc = fitz.open(pdf_path)

    categories_text = {
        "RECITAL": [],
        "ARTICLE": [],
        "ANNEX": []
    }

    noise_pattern = re.compile(r"(\d+/144|ELI: http://data\.europa\.eu/eli/reg/.+|OJ L, 12\.7\.2024|EN|\(Text with EEA relevance\))")
    footnote_cleaner = re.compile(r"\n\s*\(\s*\d+\s*\)\s*(?:OJ\b|Position\b|Directive\b|Regulation\b|See\b|Case\b).*$", re.IGNORECASE | re.DOTALL)

    footnotes_removed = 0

    for page_num in range(len(doc)):
        text = doc[page_num].get_text("text").replace("-\n", "")

        clean_lines = []
        for line in text.split("\n"):
            cleaned_line = noise_pattern.sub("", line).strip()
            if cleaned_line and not cleaned_line.isdigit():
                clean_lines.append(cleaned_line)

        # Join the page back together
        final_page_text = "\n" + "\n".join(clean_lines)

        if footnote_cleaner.search(final_page_text):
            # Replace the footnote block with a single newline
            final_page_text = footnote_cleaner.sub("\n", final_page_text)
            footnotes_removed += 1

        p_idx = page_num + 1
        if p_idx < 44:
            categories_text["RECITAL"].append(final_page_text)
        elif 44 <= p_idx <= 123:
            categories_text["ARTICLE"].append(final_page_text)
        else:
            categories_text["ANNEX"].append(final_page_text)

    doc.close()
    print(f"Removed footnote blocks from {footnotes_removed} pages.")

    units = []

    # proces recitals
    recital_text = "\n" + "\n".join(categories_text["RECITAL"]) + "\n"
    rec_parts = re.split(r"\n\s*\(\s*(\d+)\s*\)\s+", recital_text)
    for i in range(1, len(rec_parts), 2):
        text_content = rec_parts[i+1].replace("\n", " ").strip()
        if text_content:
            units.append({"id": f"RECITAL {rec_parts[i]}", "category": "RECITAL", "text": text_content})

    # proces articles
    article_text = "\n" + "\n".join(categories_text["ARTICLE"]) + "\n"
    art_parts = re.split(r"(?i)\n\s*Article\s+(\d+)\s*\n", article_text)
    for i in range(1, len(art_parts), 2):
        text_content = art_parts[i+1].replace("\n", " ").strip()
        if text_content:
            units.append({"id": f"ARTICLE {art_parts[i]}", "category": "ARTICLE", "text": text_content})

    # proces annexes
    annex_text = "\n" + "\n".join(categories_text["ANNEX"]) + "\n"
    annex_parts = re.split(r"(?i)\n\s*ANNEX\s+([IVXLCDM]+)\s*\n", annex_text)
    for i in range(1, len(annex_parts), 2):
        text_content = annex_parts[i+1].replace("\n", " ").strip()
        if text_content:
            units.append({"id": f"ANNEX {annex_parts[i].upper()}", "category": "ANNEX", "text": text_content})

    return pd.DataFrame(units)

In [ ]:
df_units = parse_ai_act_robust(local_pdf_path)
df_units['len'] = df_units['text'].str.len()

print(f"Clean Units created: {len(df_units)}")
print(f"Max length: {df_units['len'].max()}")

Removed footnote blocks from 23 pages.
Clean Units created: 307
Max length: 17095


An embedding vector represents the "average" meaning of a text. If a chunk is too long and covers five different legal requirements, the vector becomes vague, making it harder for the system to find that specific information during a search. Even if an LLM can read a huge chunk, providing too much irrelevant text alongside the answer creates "noise." This increases the risk of the model hallucinating or missing the specific detail you're looking for. Later on the lenght of the chunks will be calibrated, below is shown a visual inspection of the actual distribution.

In [ ]:
avg_len = df_units['len'].mean()

fig = go.Figure()

fig.add_trace(go.Histogram(
    x=df_units['len'],
    nbinsx=50,
    marker_color='#e74c3c',
    marker_line=dict(color='white', width=1),
    name='Unit Lengths'
))

fig.add_vline(x=1500, line_dash="dash", line_color="#c0392b")

fig.add_trace(go.Scatter(
    x=[None], y=[None],
    mode='lines',
    line=dict(color='#c0392b', dash='dash'),
    name='Target RAG Limit (1500)'
))

fig.add_vline(x=avg_len, line_dash="solid", line_color="#f39c12")

fig.add_trace(go.Scatter(
    x=[None], y=[None],
    mode='lines',
    line=dict(color='#f39c12', shape='linear'),
    name=f'Average Length ({avg_len:.0f})'
))

fig.update_layout(
    title='<b>Distribution of Legal Unit Lengths </b>',
    xaxis_title="Characters",
    yaxis_title="Number of Units",
    template='plotly_white',
    bargap=0.05,
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99,
        bgcolor="rgba(255, 255, 255, 0.5)"
    ),
    height=500
)


fig.show()

# METADATA ENRICHMENT WITH ERROR ANALYSIS

**NER FOR ENRICHING THE METADATA**


In a standard RAG system, the database relies entirely on dense vector embeddings to find relevant information. While vectors are excellent at understanding the semantic meaning or general theme of a question, they struggle significantly with exact keyword matching. For example, a vector search might easily confuse the legal obligations of a "Provider" with those of a "Deployer" because both concepts occupy a very similar mathematical space in the vector database.

**ERROR ANALYSIS METHODOLOGY**


Identifying and extracting these entities requires an iterative methodology that we'll call Error Analysis, as relying entirely on pre-trained machine learning models, this case of domain-specific legal text will lead to systemic failures. Pre-trained NLP models calculate statistical probabilities based on everyday internet text, they get easily confused by the dense capitalization, roman numerals, and acronyms found in the AI Act.

To overcome this, we established an iterative feedback loop:


1.   Baseline Definition: We injected a baseline set of deterministic rules (the EntityRuler) into the pipeline before the machine learning component.
2.   Visual Diagnostics: We ran random samples of the text through the pipeline and used a visualizer (displaCy) to inspect the model's behavior.
3. Identifying False Positives and Negatives: By reading the annotated text, we spotted areas where the model missed critical terms.
4. Rule Calibration: We updated our deterministic rules to either explicitly classify these edge cases (e.g. assigning "Annexes" to DOCUMENT)


The rules that drive this extraction are defined in a simple but highly powerful list called patterns. Every rule is a dictionary containing two key-value pairs: the "label" (which dictates the category, like "ORG" or "RISK_LEVEL") and the "pattern" (which defines what text to look for).

Afther the pattern have been identified they are added to the document object of langchain library, since could be useful to exploit these information while retriving, possibly increasing the rag system performance.

In [ ]:
nlp = spacy.load("en_core_web_sm")

ruler = nlp.add_pipe("entity_ruler", before="ner")

patterns = [
    # organizations and authoroties
    {"label": "ORG", "pattern": "AI Office"},
    {"label": "ORG", "pattern": "European Artificial Intelligence Board"},
    {"label": "ORG", "pattern": "Market Surveillance Authority"},
    {"label": "ORG", "pattern": [{"lower": "european"}, {"lower": "commission"}]},
    {"label": "ORG", "pattern": "European Data Protection Board"},
    {"label": "ORG", "pattern": "European Data Protection Supervisor"},
    {"label": "ORG", "pattern": "EDPB"},
    {"label": "ORG", "pattern": "EDPS"},
    {"label": "ORG", "pattern": "Board"},
    {"label": "ORG", "pattern": "Commission"},

    # geopolitical entities
    {"label": "GPE", "pattern": "Union"},
    {"label": "GPE", "pattern": "EU"},

    # documents
    {"label": "DOCUMENT", "pattern": "Regulation"},
    {"label": "DOCUMENT", "pattern": "Directive"},
    {"label": "DOCUMENT", "pattern": [{"lower": "annex"}, {"IS_ALPHA": True, "IS_UPPER": True}]},
    {"label": "DOCUMENT", "pattern": [{"lower": "annexes"}, {"IS_ALPHA": True, "IS_UPPER": True}]},

    # Chapters
    {"label": "LAW", "pattern": [{"lower": "chapter"}, {"IS_DIGIT": True}]},
    {"label": "LAW", "pattern": [{"lower": "chapter"}, {"IS_ALPHA": True, "IS_UPPER": True}]},

    # Sections
    {"label": "LAW", "pattern": [{"lower": "section"}, {"IS_DIGIT": True}]},
    {"label": "LAW", "pattern": [{"lower": "section"}, {"IS_ALPHA": True, "IS_UPPER": True}]},
    {"label": "LAW", "pattern": [{"lower": "sections"}, {"IS_DIGIT": True}]},
    {"label": "LAW", "pattern": [{"lower": "sections"}, {"IS_ALPHA": True, "IS_UPPER": True}]},

    # Articles
    {"label": "LAW", "pattern": [{"lower": "article"}, {"IS_DIGIT": True}]},

    # Domain Stop Words
    {"label": "IGNORE", "pattern": "AI"},
    {"label": "IGNORE", "pattern": [{"lower": "artificial"}, {"lower": "intelligence"}]},
    {"label": "IGNORE", "pattern": [{"lower": "ai"}, {"lower": "system"}]},

    # concepts and legal roles
    {"label": "CONCEPT", "pattern": "CE"},
    {"label": "CONCEPT", "pattern": [{"text": "CE"}, {"lower": "marking"}]},
    {"label": "CONCEPT", "pattern": [{"lower": "general"}, {"IS_PUNCT": True, "OP": "?"}, {"lower": "purpose"}, {"lower": "ai"}]},
    {"label": "LEGAL_ROLE", "pattern": [{"lower": "provider"}]},
    {"label": "LEGAL_ROLE", "pattern": [{"lower": "providers"}]},
    {"label": "LEGAL_ROLE", "pattern": [{"lower": "deployer"}]},
    {"label": "LEGAL_ROLE", "pattern": [{"lower": "deployers"}]},
    # risk levels
    {"label": "RISK_LEVEL", "pattern": [{"lower": "high"}, {"IS_PUNCT": True, "OP": "?"}, {"lower": "risk"}]},
    {"label": "RISK_LEVEL", "pattern": [{"lower": "unacceptable"}, {"lower": "risk"}]},
    {"label": "RISK_LEVEL", "pattern": [{"lower": "limited"}, {"lower": "risk"}]},
    {"label": "RISK_LEVEL", "pattern": [{"lower": "minimal"}, {"lower": "risk"}]},
]

ruler.add_patterns(patterns)

**LANGCHAIN DOCOMUNENT OBJECT EXPANSION**


LangChain wraps every chunk of the text into a specific Python object called a Document, it has exactly two parts:


1.   page_content: A single string of text
2.   metadata: A standard Python Dictionary

RecursiveCharacterTextSplitter() slices the text into chunks of up to 1500 characters, maintaining a 200-character overlap to ensure no nuanced legal context is lost at the boundaries. It applies "contextual chunking" by prepending the exact legal identifier (such as [ARTICLE 10]) directly into the text of every single slice, this guarantees that the LLM always knows exactly which part of the law it is referencing. Then it packages each segment into a LangChain Document object and enriches it with highly structured metadata, including the category, legal unit, and a flag indicating if it is a continuation resulting in a robust, searchable dataset ready for embedding into the FAISS vector database.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", ".", " ", ""]
)

final_documents = []


for _, row in df_units.iterrows():
    unit_id = row['id']
    category = row['category']
    text = row['text']

    chunks = text_splitter.split_text(text)

    for i, chunk in enumerate(chunks):
        enriched_content = f"[{unit_id}] {chunk}"

        combined_metadata = {
            "source": "EU_AI_Act_2024",
            "category": category,
            "legal_unit": unit_id,
            "chunk_index": i,
            "is_continuation": i > 0
        }

        spacy_doc = nlp(enriched_content)
        extracted = defaultdict(set)

        for ent in spacy_doc.ents:
            if ent.label_ != "IGNORE":
                extracted[ent.label_].add(ent.text)

        for label, entity_set in extracted.items():
            combined_metadata[f"ner_{label}"] = list(entity_set)

        doc = Document(
            page_content=enriched_content,
            metadata=combined_metadata
        )

        final_documents.append(doc)

print(f"Total vectorized chunks ready for FAISS: {len(final_documents)}")

Total vectorized chunks ready for FAISS: 577


A quick check of the information a single chunk is holding:

In [ ]:
sample_doc = random.choice(final_documents)

for key, value in sample_doc.metadata.items():
    if not key.startswith("ner_"):
        clean_key = key.replace('_', ' ').title()
        print(f"{clean_key}: {value}")


found_entities = False
for key, value in sample_doc.metadata.items():
    if key.startswith("ner_"):
        found_entities = True
        print(f"{key}: {value}")

if not found_entities:
    print("(No entities found in this specific chunk)")

print("\n--- Text Preview ---")
print(f"{sample_doc.page_content[:800]}...")

Source: EU_AI_Act_2024
Category: ARTICLE
Legal Unit: ARTICLE 10
Chunk Index: 1
Is Continuation: True
ner_LAW: ['ARTICLE 10']
ner_CARDINAL: ['2.', '3.']
ner_RISK_LEVEL: ['high-risk']
ner_GPE: ['Union']
ner_DOCUMENT: ['Regulation']

--- Text Preview ---
[ARTICLE 10] 2. Training, validation and testing data sets shall be subject to data governance and management practices appropriate for the intended purpose of the high-risk AI system. Those practices shall concern in particular: (a) the relevant design choices; (b) data collection processes and the origin of data, and in the case of personal data, the original purpose of the data collection; (c) relevant data-preparation processing operations, such as annotation, labelling, cleaning, updating, enrichment and aggregation; (d) the formulation of assumptions, in particular with respect to the information that the data are supposed to measure and represent; (e) an assessment of the availability, quantity and suitability of the data sets that

An additional refinement step was implemented to optimize the dataset for the Retrieval-Augmented Generation architecture. Following the initial text splitting, a filtering mechanism was applied to remove 7 low-value, boilerplate chunks, specifically legislative preambles and fragmented text containing fewer than 100 characters. Addiotionaly, leading punctuation artifacts generated by the recursive splitting process were stripped. This reduction from 577 to 570 highly concentrated text chunks ensures the FAISS vector database is populated exclusively with semantically dense legal provisions. By eliminating contextual noise, this approach maximizes embedding precision, reduces retrieval latency, and mitigates the risk of LLM hallucinations during the generation phase.

In [ ]:
polished_documents = []
deleted_chunks = []

for doc in final_documents:
    clean_text = re.sub(r"^(\[.*?\])\s*[,.]+\s*", r"\1 ", doc.page_content)

    if len(clean_text) > 100:
        doc.page_content = clean_text
        polished_documents.append(doc)
    else:
        doc.page_content = clean_text
        deleted_chunks.append(doc)

print(f"Original chunks: {len(final_documents)}")
print(f"Polished chunks ready for vectorization: {len(polished_documents)}")
print(f"Removed {len(deleted_chunks)} low-value chunks.\n")

Original chunks: 577
Polished chunks ready for vectorization: 570
Removed 7 low-value chunks.



In [ ]:
print(" DELETED CHUNKS PREVIEW ")
if not deleted_chunks:
    print("No chunks were deleted.")
else:
    for i, doc in enumerate(deleted_chunks):
        print(f"\n--- Deleted Chunk {i+1} ---")
        print(f"Legal Unit: {doc.metadata.get('legal_unit', 'Unknown')}")
        print(f"Length: {len(doc.page_content)} characters")
        print(f"Text: '{doc.page_content}'")

 DELETED CHUNKS PREVIEW 

--- Deleted Chunk 1 ---
Legal Unit: RECITAL 8
Length: 12 characters
Text: '[RECITAL 8] '

--- Deleted Chunk 2 ---
Legal Unit: RECITAL 35
Length: 31 characters
Text: '[RECITAL 35] Such data includes'

--- Deleted Chunk 3 ---
Legal Unit: ARTICLE 3
Length: 12 characters
Text: '[ARTICLE 3] '

--- Deleted Chunk 4 ---
Legal Unit: ARTICLE 5
Length: 37 characters
Text: '[ARTICLE 5] Prohibited AI practices 1'

--- Deleted Chunk 5 ---
Legal Unit: ARTICLE 13
Length: 13 characters
Text: '[ARTICLE 13] '

--- Deleted Chunk 6 ---
Legal Unit: ARTICLE 16
Length: 13 characters
Text: '[ARTICLE 16] '

--- Deleted Chunk 7 ---
Legal Unit: ARTICLE 66
Length: 13 characters
Text: '[ARTICLE 66] '


Running displacy one final time on the polished RAG chunks to prove that the custom entities defined in PATTERNS survived the text splitting, metadata injection, and string cleaning processes without being corrupted, and showcasing the accuracy reached by the ML model to identify entity from the AI act.

In [ ]:
sample_chunks = random.sample(polished_documents, 3)

for i, doc in enumerate(sample_chunks):
    print(f"\n--- CHUNK {i+1} (Source: {doc.metadata['legal_unit']}) ---")
    spacy_doc = nlp(doc.page_content)

    custom_colors = {
        "LEGAL_ROLE": "linear-gradient(90deg, #f1c40f, #f39c12)",
        "RISK_LEVEL": "linear-gradient(90deg, #e74c3c, #c0392b)",
        "CONCEPT": "linear-gradient(90deg, #3498db, #2980b9)",
        "DOCUMENT": "linear-gradient(90deg, #9b59b6, #8e44ad)",
        "LAW": "linear-gradient(90deg, #34495e, #2c3e50)",
        "GPE": "linear-gradient(90deg, #2ecc71, #27ae60)"
    }

    #  hides the 'IGNORE' tags
    visible_ents = ["ORG", "GPE", "DOCUMENT", "LAW", "CONCEPT", "LEGAL_ROLE", "RISK_LEVEL"]

    options = {
        "colors": custom_colors,
        "ents": visible_ents
    }

    displacy.render(spacy_doc, style="ent", jupyter=True, options=options)


--- CHUNK 1 (Source: ARTICLE 60) ---



--- CHUNK 2 (Source: ARTICLE 99) ---



--- CHUNK 3 (Source: ARTICLE 101) ---


**ENTITY TYPE DISTRIBUTION**


Not all extracted metadata provide the same level of value for retrieval and reasoning. Some fields are relatively generic or structurally redundant and may have limited impact on ranking performance, while others are particularly informative in a legal and regulatory domain. In particular, risk levels, explicit legal references (e.g. articles and paragraphs), and regulatory actors carry strong semantic and normative signals and can significantly improve filtering, disambiguation, and answer grounding in a RAG system. Nevertheless, all extracted metadata are retained, as they may become useful for future extensions.

In [ ]:
all_entity_counts = Counter()

for doc in polished_documents:
    for key, val_list in doc.metadata.items():
        if key.startswith("ner_"):
            clean_label = key.replace("ner_", "")
            all_entity_counts[clean_label] += len(val_list)

label_df = pd.DataFrame(all_entity_counts.most_common(), columns=['Entity Type', 'Count'])

fig = px.bar(
    label_df,
    x='Entity Type',
    y='Count',
    title='Named Entity Type Distribution (All Entities)',
    color='Count',
    color_continuous_scale='RdBu_r',
    text_auto=True
)

fig.update_layout(
    xaxis_title='Entity Type',
    yaxis_title='Frequency',
    xaxis_tickangle=-45,
    template='plotly_white',
    showlegend=False
)

fig.show()

Finally the chunks are ready and is possible to check the final lenght distribution.

In [ ]:
avg_len = sum(chunk_lengths) / len(chunk_lengths) if chunk_lengths else 0

fig = go.Figure()

fig.add_trace(go.Histogram(
    x=chunk_lengths,
    nbinsx=50,
    marker_color='#e74c3c',
    marker_line=dict(color='white', width=1),
    name='Chunk Lengths'
))

fig.add_vline(x=1500, line_dash="dash", line_color="#c0392b")

fig.add_trace(go.Scatter(
    x=[None], y=[None],
    mode='lines',
    line=dict(color='#c0392b', dash='dash'),
    name='Max Chunk Limit (1500)'
))

fig.add_vline(x=avg_len, line_dash="solid", line_color="#f39c12")


fig.add_trace(go.Scatter(
    x=[None], y=[None],
    mode='lines',
    line=dict(color='#f39c12'),
    name=f'Average Length ({avg_len:.0f})'
))

fig.update_layout(
    title='<b>Distribution of Final Vectorized Chunk Lengths</b>',
    xaxis_title="Characters per Chunk",
    yaxis_title="Number of Chunks",
    template='plotly_white',
    bargap=0.05,
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01,
        bgcolor="rgba(255, 255, 255, 0.5)"
    ),
    height=500
)


fig.show()

Below is shown a barplot of the absolute count of chunks belonging to RECITAL, ARTICLE, and ANNEX. This proves that the dataset is heavily weighted toward operative legal text (Articles) but retains the contextual intent (Recitals).

In [ ]:
categories = [doc.metadata['category'] for doc in polished_documents]
category_counts = Counter(categories)
sorted_categories = sorted(category_counts.items(), key=lambda x: x[1], reverse=True)

labels = [x[0] for x in sorted_categories]
values = [x[1] for x in sorted_categories]

fig = go.Figure(data=[
    go.Bar(
        x=labels,
        y=values,
        text=values,
        textposition='auto',
        marker=dict(
            color=values,
            colorscale=[
                [0, '#ffcccc'],
                [1, '#b30000']
            ],
            showscale=False,
            line=dict(color='white', width=1.5)
        )
    )
])

fig.update_layout(
    title='<b>Document Structural Balance</b><br><sup>Chunks per Category</sup>',
    xaxis_title="Category",
    yaxis_title="Number of Vectorized Chunks",
    template='plotly_white',
    font=dict(family="Arial", size=12),
    bargap=0.2,
    height=500
)


fig.show()

# TF-IDF EDA

The TF-IDF approach is an essential exploratory tool because it goes far beyond simple word counts, which can often be misleading. TF-IDF evaluates the importance of a word by balancing its local frequency within a specific chunk against its global rarity across the entire dataset.  It deliberately penalizes common words that appear everywhere and assigns greater weight to terms that uniquely characterize individual documents. This mathematical filtering ensures that words with higher TF-IDF scores genuinely represent the most significant and meaningful concepts of the text.

In [ ]:
corpus = [doc.page_content for doc in polished_documents]

custom_stop_words = ['shall', 'article', 'regulation', 'paragraph', 'annex',
                     'member', 'states', 'union', 'eu', 'referred', 'accordance', 'chapter', 'recital']

all_stop_words = list(ENGLISH_STOP_WORDS) + custom_stop_words

vectorizer = TfidfVectorizer(stop_words=all_stop_words, max_features=1000)
tfidf_matrix = vectorizer.fit_transform(corpus)

word_scores = tfidf_matrix.sum(axis=0).A1
words = vectorizer.get_feature_names_out()

df_words = pd.DataFrame({'word': words, 'score': word_scores})
top_20_words = df_words.sort_values(by='score', ascending=False).head(20)

top_20_words = top_20_words.sort_values(by='score', ascending=True)

fig = go.Figure()

fig.add_trace(go.Bar(
    y=top_20_words['word'],
    x=top_20_words['score'],
    orientation='h',
    marker=dict(
        color='#9b59b6',
        line=dict(color='white', width=1)
    ),
    name='TF-IDF Score'
))

fig.update_layout(
    title='<b>Top 20 Most Important Terms in the AI Act (TF-IDF)</b>',
    xaxis_title="Cumulative TF-IDF Score",
    yaxis_title="Terms",
    template='plotly_white',
    height=700,
    yaxis={'categoryorder':'total ascending'},
    margin=dict(l=150)
)

fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='rgba(0,0,0,0.1)')

fig.show()

# EMBEDDDING THE CHUNKS AND CREATION OF DATABASE

In [ ]:
!pip install sentence-transformers faiss-cpu

**BGE_M3**


It uses a shared vector space for over 100 languages. This is critical for the AI Act, as it allows a user to query in their native tongue and retrieve the precise corresponding legal section, even if the primary knowledge base is in English, BGE-M3 demonstrates state-of-the-art cross-lingual transfer. It captures the nuanced "legal spirit" of a clause across different linguistic structures without losing the technical accuracy of the definition.
By setting normalize_embeddings=True, the model's ability to weight important legal keywords in enhanced. In the context of the AI Act, this means the model "learns" that terms like "High-Risk" are more vital to the search result than common stop-words

In [ ]:
hf_embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs={'device': 'cuda'},
    encode_kwargs={'normalize_embeddings': True}
)
print("Model loaded")

print(f"Embedding {len(polished_documents)} chunks. This will take a moment...")

vectorstore = FAISS.from_documents(polished_documents, hf_embeddings) # text to gpu, vectorizes, and organizes in FAISS index
print("FAISS index built")

vectorstore.save_local(drive_persistence_path)

print("\n Vector database created and saved")

# RETRIVAL LOGIC

The following code implements a search architecture using LangChain's EnsembleRetriever to overcome the individual limitations of semantic and keyword-based retrieval methods. While the standard FAISS vector search (dense retrieval) excels at understanding the underlying contextual meaning of a query, it can occasionally miss exact-match requirements, such as specific legal terminology or article numbers. Conversely, the BM25 algorithm (sparse retrieval) is highly precise for exact keyword matching but completely lacks semantic comprehension.

By combining both approaches with equal influence (weights=[0.5, 0.5]), the system executes both searches in parallel and merges their outputs using Reciprocal Rank Fusion (RRF). This mathematical algorithm recalculates the score of each document based on its rank position in both individual lists. Consequently, legal chunks that contain both the exact terminology (captured by BM25) and the correct contextual meaning (captured by FAISS) are elevated to the top of the final results. This dual-engine approach drastically increases overall retrieval precision, ensuring the generative AI model is grounded in the most accurate and comprehensive legal context possible.

In [ ]:
!pip install rank_bm25

In [ ]:
# instanciating the model and loading database
hf_embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs={'device': 'cuda'},
    encode_kwargs={'normalize_embeddings': True}
)


vectorstore = FAISS.load_local(
    folder_path=drive_persistence_path,
    embeddings=hf_embeddings,
    allow_dangerous_deserialization=True
)
print("Database loaded")

In [ ]:
all_possible_ner_keys = set()
for doc in polished_documents:
    for key in doc.metadata.keys():
        if key.startswith("ner_"):
            all_possible_ner_keys.add(key)

all_possible_ner_keys = sorted(list(all_possible_ner_keys))

query = "What are the rules and obligations for high-risk biometric identification systems?"
print(f"User Question: '{query}'\n")

bm25_retriever = BM25Retriever.from_documents(polished_documents)
bm25_retriever.k = 3

faiss_retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

hybrid_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever],
    weights=[0.5, 0.5] # 50% keyword, 50% semantic
)

top_chunks = hybrid_retriever.invoke(query)[:5]


for i, chunk in enumerate(top_chunks):
    print(f"RESULT {i+1} ")

    for key, value in chunk.metadata.items():
        if not key.startswith("ner_"):
            clean_key = key.replace('_', ' ').title()
            print(f" - {clean_key}: {value}")

    print(f"\nText Preview: {chunk.page_content[:500]}...\n")

    for ner_key in all_possible_ner_keys:
        display_name = ner_key.replace('ner_', '')
        extracted_entities = chunk.metadata.get(ner_key, [])
        print(f" - {display_name}: {extracted_entities}")
    print("\n")

User Question: 'What are the rules and obligations for high-risk biometric identification systems?'

RESULT 1 
 - Source: EU_AI_Act_2024
 - Category: RECITAL
 - Legal Unit: RECITAL 3
 - Chunk Index: 1
 - Is Continuation: True

Text Preview: [RECITAL 3] To the extent that this Regulation contains specific rules on the protection of individuals with regard to the processing of personal data concerning restrictions of the use of AI systems for remote biometric identification for the purpose of law enforcement, of the use of AI systems for risk assessments of natural persons for the purpose of law enforcement and of the use of AI systems of biometric categorisation for the purpose of law enforcement, it is appropriate to base this Regu...

 - CARDINAL: []
 - CONCEPT: []
 - DATE: []
 - DOCUMENT: ['Regulation']
 - EVENT: []
 - FAC: []
 - GPE: []
 - LAW: ['Article 16']
 - LEGAL_ROLE: []
 - LOC: []
 - NORP: []
 - ORDINAL: []
 - ORG: ['European Data Protection Board']
 - PERCENT: []
 - PERSON: 

**HYBRID RETRIVAL (METADATA-FILTERED)**

Standard search relies strictly on mathematical scoring (vector proximity and keyword frequency) to rank documents. While highly effective at finding general contextual relevance, it cannot guarantee that the retrieved documents adhere to specific factual constraints, such as ensuring a legal obligation strictly applies to a "deployer" rather than a "provider".
To solve this, the code below broadens the initial search space by retrieving the top 15 results from both the dense and sparse retrievers. Once the Reciprocal Rank Fusion algorithm merges and mathematically ranks these chunks into a list, the loop evaluates the pre-extracted Named Entity Recognition tags attached to each chunk's metadata. Any chunk that does not explicitly contain the target entity ("deployer") is immediately discarded, regardless of how high its mathematical search score was.
This architecture increases precision by combining the semantic intelligence of a standard search with the rigid, rules-based logic of metadata filtering. It eliminates the risk of "entity confusion" ensuring the generative LLM only reads legal rules that definitively apply to the exact stakeholder requested in the prompt.

In [ ]:
query = "What are the registration obligations for high-risk biometric systems?"

bm25_retriever.k = 15
faiss_retriever.search_kwargs = {"k": 15}

raw_hybrid_chunks = hybrid_retriever.invoke(query)

filtered_chunks = []
for chunk in raw_hybrid_chunks:
    legal_roles = chunk.metadata.get("ner_LEGAL_ROLE", [])

    if any("deployer" in role.lower() for role in legal_roles):
        filtered_chunks.append(chunk)

if not filtered_chunks:
    print("No chunks found with 'deployer' metadata.")


for i, chunk in enumerate(filtered_chunks):
    print(f"FILTERED HYBRID RESULT {i+1} ")

    for key, value in chunk.metadata.items():
        if not key.startswith("ner_"):
            clean_key = key.replace('_', ' ').title()
            print(f" - {clean_key}: {value}")

    print("\nTarget Metadata Found:")
    print(f" - Legal Roles: {chunk.metadata.get('ner_LEGAL_ROLE', [])}")

    print(f"\nText Preview: {chunk.page_content[:500]}...\n")

bm25_retriever.k = 3
faiss_retriever.search_kwargs = {"k": 3}

FILTERED HYBRID RESULT 1 
 - Source: EU_AI_Act_2024
 - Category: ARTICLE
 - Legal Unit: ARTICLE 26
 - Chunk Index: 3
 - Is Continuation: True

Target Metadata Found:
 - Legal Roles: ['Deployers', 'deployer', 'deployers', 'provider']

Text Preview: [ARTICLE 26] 8. Deployers of high-risk AI systems that are public authorities, or Union institutions, bodies, offices or agencies shall comply with the registration obligations referred to in Article 49. When such deployers find that the high-risk AI system that they envisage using has not been registered in the EU database referred to in Article 71, they shall not use that system and shall inform the provider or the distributor. 9. Where applicable, deployers of high-risk AI systems shall use t...

FILTERED HYBRID RESULT 2 
 - Source: EU_AI_Act_2024
 - Category: ARTICLE
 - Legal Unit: ARTICLE 26
 - Chunk Index: 4
 - Is Continuation: True

Target Metadata Found:
 - Legal Roles: ['deployer']

Text Preview: [ARTICLE 26] Each use shall be limite

# DISTILBERT CLASSIFIER

The following is the prompt used to generate the synthetic dataset, with the help of this LLM: Gemini 3 (reasoning)

The prompt:
You are an expert AI data engineer and EU legal scholar. I am fine-tuning a DistilBERT text classifier and need a highly diverse synthetic dataset of AI product descriptions mapped to their official EU AI Act risk levels.
Generate exactly 100 rows of data in perfect CSV format.
The Rules:

The CSV must have exactly two columns: text and label.
The text column should be a 1 to 2 sentence description of a fictional AI system or startup product. Enclose every text description in double quotes ("").
The label column must be EXACTLY one of these four strings (no spaces, case-sensitive): Unacceptable_Risk, High_Risk, Limited_Risk, Minimal_Risk.
Generate exactly 25 examples for each of the 4 categories.
Crucial: Maximize industry diversity! Do not just use medical or HR examples. Invent systems for agriculture, toys, deepfakes, law enforcement, education, banking, video games, aviation, and chatbots.
Do not include any markdown formatting like ```csv or conversational text in your response. Output ONLY the raw CSV text starting with the header row, so I can copy and paste it directly.

In [ ]:
!pip install transformers datasets evaluate accelerate -U

Synthetic dataset loading:

In [6]:
file_path = "/content/drive/MyDrive/compliance_and_regulation_rag/ai_risk_dataset.csv.txt"

df = pd.read_csv(file_path)
print(df.head())
print(df['label'].value_counts())

                                                text              label
0  An AI-powered social scoring system used by mu...  Unacceptable_Risk
1  A real-time biometric identification system de...  Unacceptable_Risk
2  An AI application designed to subconsciously m...  Unacceptable_Risk
3  A predictive policing tool used to identify an...  Unacceptable_Risk
4  A cognitive-behavioral manipulation AI used by...  Unacceptable_Risk
label
Unacceptable_Risk    75
High_Risk            75
Limited_Risk         75
Minimal_Risk         75
Name: count, dtype: int64


Defining train, val, test split, performance metrics and training (fine-tuning) parameters:

In [ ]:
label_map = {"Minimal_Risk": 0, "Limited_Risk": 1, "High_Risk": 2, "Unacceptable_Risk": 3}
df['label'] = df['label'].map(label_map)

train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
eval_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])

train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)
test_dataset = Dataset.from_pandas(test_df)

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_eval = eval_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=4)

def compute_metrics(eval_pred):
    logits, labels = eval_pred

    predictions = np.argmax(logits, axis=-1)

    probabilities = softmax(logits, axis=-1)

    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='macro')
    recall = recall_score(labels, predictions, average='macro')

    auc = roc_auc_score(labels, probabilities, multi_class='ovr', average='macro')

    # nir
    largest_class_count = max(np.bincount(labels))
    nir = largest_class_count / len(labels)

    return {
        "accuracy": accuracy,
        "f1_macro": f1,
        "recall_macro": recall,
        "roc_auc": auc,
        "NIR": nir
    }

training_args = TrainingArguments(
    output_dir="./ai_risk_classifier",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=6,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    compute_metrics=compute_metrics,
)

In [9]:
print(f"Train: {len(train_df)} | Val: {len(eval_df)} | Test: {len(test_df)}")

Train: 240 | Val: 30 | Test: 30


Training and saving the best model (based on validation loss)

In [ ]:
trainer.train()

print("\n=== test set evaluation ===")
test_results = trainer.predict(tokenized_test)

trainer.save_model("./saved_ai_risk_model")
tokenizer.save_pretrained("./saved_ai_risk_model")
print("\nTraining complete, Architecture saved.")

In [12]:
log_history = trainer.state.log_history

train_loss = []
train_epochs = []
eval_loss = []
eval_epochs = []


for entry in log_history:
    if "loss" in entry and "epoch" in entry:
        train_loss.append(entry["loss"])
        train_epochs.append(entry["epoch"])
    elif "eval_loss" in entry and "epoch" in entry:
        eval_loss.append(entry["eval_loss"])
        eval_epochs.append(entry["epoch"])

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=train_epochs,
    y=train_loss,
    mode='lines+markers',
    name='Training Loss',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=eval_epochs,
    y=eval_loss,
    mode='lines+markers',
    name='Validation Loss',
    line=dict(color='orange')
))

fig.update_layout(
    title='Training vs. Validation Loss',
    xaxis_title='Epochs',
    yaxis_title='Loss',
    template='plotly_white',
    hovermode='x unified')

fig.show()

Performance report of DISTILBERT:

In [ ]:
print(f"Accuracy: {test_results.metrics['test_accuracy'] * 100:.2f}%")
print(f"F1 Score: {test_results.metrics['test_f1_macro']:.4f}")
print(f"Recall:   {test_results.metrics['test_recall_macro']:.4f}")
print(f"ROC AUC:  {test_results.metrics['test_roc_auc']:.4f}")
print(f"NIR:      {test_results.metrics['test_NIR'] * 100:.2f}%")

Accuracy: 90.00%
F1 Score: 0.8995
Recall:   0.8973
ROC AUC:  0.9747
NIR:      26.67%


# GIT PUSH

In [ ]:
# for pushing
!git add .
!git commit -m "env created"
!git push -u origin main

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
Branch 'main' set up to track remote branch 'main' from 'origin'.
Everything up-to-date
